In [38]:
import pandas as pd
df_dict = pd.read_excel("FutureBookings.xlsx", sheet_name=None) 
print(df_dict.keys())  
new_df = df_dict['TicketBookings']
new_df.tail() 

dict_keys(['Masters', 'TicketBookings'])


,Starting Point,Destination,Booked Before Days,Month of Travel,Number of travellers,Number of previous cancellations,Festival,Class of Train,Class of Ticket,State of Travellers,Weekday of Travel
10584,Bangalore,Mumbai CST,1,November,4,2,NaN,Superfast,Second AC 2 Tier,Goa,Wedensday
10585,Ernakulam,Ratnagiri,4,January,1,2,NaN,Rajadhani,Second Ordinary,Karnataka,Wedensday
10586,Kannur,Vasai Road,29,January,6,0,Pongal,Superfast,Second AC 2 Tier,Gujarat,Sunday
10587,Ernakulam,Roha,23,January,6,1,Pongal,Express,Second AC 3 Tier,Tamilnadu,Sunday
10588,Mookambika Road,Bangalore,28,January,2,0,Pongal,Jan Satabdi,Second AC 3 Tier,Gujarat,Monday


In [44]:
new_df.shape

(10589, 12)

In [45]:
import pickle
import pandas as pd
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

with open("model_ticket.pkl", "rb") as f:
    model = pickle.load(f)

with open("model_columns_of_ticket.pkl", "rb") as f:
    model_columns = pickle.load(f)

with open("scaler_of_ticket.pkl", "rb") as f:
    scaler = pickle.load(f)


#new_df = pd.read_excel("FutureBookings.xlsx")

for col in ["Month of Travel", "Class of Train", "State of Travellers"]:
    new_df[col] = new_df[col].fillna(new_df[col].mode()[0])


new_encoded = pd.get_dummies(new_df[["Month of Travel", "Class of Train", "State of Travellers"]])


new_encoded = new_encoded.reindex(columns=model_columns, fill_value=0)


new_scaled = scaler.transform(new_encoded)
predictions = model.predict(new_scaled)


new_df["Predicted Cancellation"] = ["Cancelled" if p == 1 else "Not Cancelled" for p in predictions]


print(new_df[["Month of Travel", "Class of Train", "State of Travellers", "Predicted Cancellation"]])


      Month of Travel Class of Train State of Travellers  \
0             October    Jan Satabdi           Tamilnadu   
1             January      Rajadhani                 Goa   
2            December     Garib Rath             Gujarat   
3            November        Express              Kerala   
4            December    Jan Satabdi           Tamilnadu   
...               ...            ...                 ...   
10584        November      Superfast                 Goa   
10585         January      Rajadhani           Karnataka   
10586         January      Superfast             Gujarat   
10587         January        Express           Tamilnadu   
10588         January    Jan Satabdi             Gujarat   

      Predicted Cancellation  
0              Not Cancelled  
1              Not Cancelled  
2              Not Cancelled  
3              Not Cancelled  
4                  Cancelled  
...                      ...  
10584          Not Cancelled  
10585          Not Cancelled  


In [40]:
b = new_df[new_df['Predicted Cancellation'] == 'Cancelled']
b.head()

,Starting Point,Destination,Booked Before Days,Month of Travel,Number of travellers,Number of previous cancellations,Festival,Class of Train,Class of Ticket,State of Travellers,Weekday of Travel,Predicted Cancellation
4,Alappuzha,Bhatkal,19,December,7,3,NaN,Jan Satabdi,Second AC 3 Tier,Tamilnadu,Friday,Cancelled
7,Thivim,Kottayam,20,December,5,0,NaN,Rajadhani,Second AC 3 Tier,Maharashtra,Thursday,Cancelled
13,Udupi,Kumta,8,December,6,3,Christmas,Express,Second AC Economy,Kerala,Tuesday,Cancelled
25,Mumbai LTT,Kottayam,41,November,1,3,Pongal,Rajadhani,Second AC 3 Tier,Kerala,Sunday,Cancelled
31,Tirur,Ernakulam,52,December,6,1,NaN,Rajadhani,Second AC 3 Tier,Maharashtra,Wedensday,Cancelled


In [41]:
c = new_df[new_df['Predicted Cancellation'] == 'Not Cancelled']
c.head()

,Starting Point,Destination,Booked Before Days,Month of Travel,Number of travellers,Number of previous cancellations,Festival,Class of Train,Class of Ticket,State of Travellers,Weekday of Travel,Predicted Cancellation
0,Chiplun,Mookambika Road,19,October,3,1,Christmas,Jan Satabdi,First AC Coupe,Tamilnadu,Thursday,Not Cancelled
1,Madgaon,Panvel,55,January,4,1,NaN,Rajadhani,Second AC 2 Tier,Goa,Wedensday,Not Cancelled
2,Kannur,Thrissur,22,December,4,1,NaN,Garib Rath,First AC Coupe,Gujarat,Saturday,Not Cancelled
3,Kozhikkode,Thivim,52,November,6,0,Pongal,Express,First AC,Kerala,Saturday,Not Cancelled
5,Kottayam,Kollam,17,November,7,2,Christmas,Express,First AC Coupe,Karnataka,Wedensday,Not Cancelled


In [42]:
b.shape

(1590, 12)

In [43]:
c.shape

(8999, 12)

In [46]:
sample = pd.DataFrame({
    "Month of Travel": ["July"],
    "Class of Train": ["Jan Satabdi"],
    "State of Travellers": ["Tamilnadu"]
})


In [47]:
sample_encoded = pd.get_dummies(sample)
sample_encoded = sample_encoded.reindex(columns=new_encoded.columns, fill_value=0)
sample_scaled = scaler.transform(sample_encoded)

predictions = model.predict(sample_scaled)
print("Prediction:", "Cancelled" if predictions[0] == 1 else "Not Cancelled")

Prediction: Cancelled


In [50]:
def predict_ticket_cancellation(month, train_class, state):
    import pandas as pd

    sample = pd.DataFrame({
        "Month of Travel": [month],
        "Class of Train": [train_class],
        "State of Travellers": [state]
    })

    sample_encoded = pd.get_dummies(sample)
    sample_encoded = sample_encoded.reindex(columns=new_encoded.columns, fill_value=0)

    sample_scaled = scaler.transform(sample_encoded)

    prediction = model.predict(sample_scaled)
    return "Cancelled" if prediction[0] == 1 else "Not Cancelled"


In [51]:
predict_ticket_cancellation("July", "Jan Satabdi","Tamilnadu")

'Cancelled'

In [52]:
predict_ticket_cancellation("October", "Jan Satabdi","Tamilnadu")

'Not Cancelled'

In [53]:
predict_ticket_cancellation("December", "Rajadhani","Maharashtra")

'Cancelled'